
Saadan teile failid .zip kujul. CSV failides on kaks andmetabelit, mis koosnevad sünteetilitest andmetest, mis põhinevad reaalsete vähihaigete (kopsu ja eesnäärme) haigustrajektooridel. Andmetabelitel on kolm tunnust SUBJECT_ID - unikaalne patsiendi id; DEFINITION_ID - meditsiiniline sekkumine, mis patsiendiga toimus; TIME - aeg aastates, millal sekkumine toimus. Ajal 0 sai iga patsient vähidiagnoosi. Osadel patsientidest on olemas seisund "death", mis tähendab, et see patsient suri antud ajahetkel. Kõikide patsientide puhul on kustutatud aasta enne suremist/viimast üleskirjutist.

Teie ülesanne on ennustada patsientide suremust, ehk siis klassifitseerimisülesanne, kus klass 1 - patsient sureb; 0 - patsient jääb aasta jooksul ellu. Üks viis andmeid töödelda, selleks, et neid mudelitele ette sööta, on iga unikaalne sekkumine ja selle järjekorra number muuta eraldi tunnuseks, tekitada iga patsiendi kohta üks rida ja väärtuseks võtta aeg. Kuid igasugune andmete töötlemine on teie enda vaba valik. Te võite tekitada klassifitseerija ükskõik millisel meetodil: otsustuspuud, närvivõrgud, logistiline regressioon vms. 

Üks peamine probleem terviseandmetega on tõsisasi, et need on üsna mürased. Erinevate sekkumiste all on peidetud definitsioonid, millest osad on arvatavasti ennustamiseks olulised (näiteks "keemiaravi") ja teised mitte nii väga (näiteks "nägemiskontroll"). Müraste andmete/tunnuste eemaldamine võib antud ülesande puhul suuresti tõsta mudeli täpsust. Käesoleva ülesande juures on tähtis, et te ei või eeldada, et sekkumiste definitsioonid on erinevate andmetabelite seas sama tähendusega (antud juhul näiteks, et Drug_1 on sama, mis Drug_1 teises andmestikus).

Töö tulemuseks peaks olema töövoog, mis võtab samasuguse andmetabeli sisendiks ning väljastaks AUC-ROC väärtuse ja mudeli enda. Seda võib teha nii jupyter-notebookis kui ka skripti kujul.

ZIP failist leiate kaks andmetabelit, võite üht kasutada töövoo loomiseks ja teist selleks, et seda valideerida. Andmed on sünteetilised ja pole üksteisega seotud (SUBJECT_ID väärtus 1 ühes ei tähenda, et tegu on sama inimesega teises andmetabelis). Mul endal on ligipääs päris andmetele, mida ma teiega kahjuks jagada ei saa, aga ma võin jooksutada teie töövoogu päris andmetel ja jagada teiega tulemusi. Te võite väljundisse lisada erinevat statistikat, mida soovite pärisandmetelt saada. Need väljundid peavad olema summeeritud, mis tähendab, et ma ei saa teiega jagada patsiendi tasemel andmeid.

Ma ise loodan, et teie loodud töövoog selekteeriks tunnuseid. Parim terviseandmete mudel on võimalikult väikese arvu tunnustega, samuti teades neid tunnuseid saab edendada teadustööd nii haiguste kui ka nende ärahoidmise/ravimise kohta. Samuti soovitan julgelt proovida graafi loomist ning sellelt "key-playerite" jms leidmist, see oleks innovaatiline viis tunnuste valimiseks.

FYI, neid samu andmeid kasutades sain ma juhumetsaga AUCROC'i 0.65 ja pärast tunnuste selekteerimist oli AUCROC 0.75. Juhul kui töö tulemus on hea ja kasutatav ka muude haiguste ennustusmudelite loomisel on võimalus saada oma nimi teadusartiklile 🙂 



In [14]:
import pandas as pd
data = pd.read_csv("synthetic_data_lung_cancer.csv")
data2 = pd.read_csv("synthetic_data_pca.csv")
pd.set_option('display.max_rows', 20)
surnud= data[data["DEFINITION_ID"] == "death"]
subject_ids = surnud["SUBJECT_ID"]
# Filter the DataFrame based on the extracted 'SUBJECT_ID' values
surnud = data[data["SUBJECT_ID"].isin(subject_ids)]
surnud["SUBJECT_ID"].value_counts()
surnud["SUBJECT_ID"]


575         2
576         2
577         2
578         2
579         2
         ... 
560903    981
560904    981
560905    981
560906    981
560907    981
Name: SUBJECT_ID, Length: 102876, dtype: int64

In [15]:
surnud["SUBJECT_ID"].unique()


array([  2,   4,  10,  12,  15,  17,  18,  22,  24,  25,  30,  35,  37,
        38,  46,  47,  48,  58,  60,  65,  70,  78,  92,  95,  98, 100,
       102, 103, 108, 109, 115, 122, 128, 130, 131, 134, 144, 149, 152,
       156, 162, 163, 170, 171, 173, 175, 176, 187, 193, 194, 205, 215,
       218, 221, 241, 245, 246, 248, 249, 251, 255, 256, 260, 264, 270,
       271, 273, 274, 276, 282, 289, 293, 296, 303, 315, 318, 320, 321,
       325, 333, 335, 336, 338, 340, 343, 345, 352, 353, 356, 357, 360,
       361, 364, 367, 369, 370, 375, 377, 378, 382, 383, 400, 401, 402,
       403, 404, 407, 413, 414, 416, 432, 433, 435, 438, 441, 444, 447,
       449, 460, 462, 464, 465, 466, 469, 474, 476, 479, 482, 491, 493,
       494, 498, 501, 502, 503, 508, 513, 515, 517, 520, 522, 524, 525,
       527, 529, 530, 533, 539, 541, 553, 557, 559, 561, 562, 565, 570,
       571, 584, 585, 586, 587, 592, 594, 608, 617, 618, 624, 626, 628,
       629, 630, 644, 645, 646, 655, 659, 660, 670, 673, 676, 67

In [16]:
data[data["SUBJECT_ID"] == 737]

,SUBJECT_ID,DEFINITION_ID,TIME
439936,737,condition_1758,0.004961
439937,737,condition_1673,0.008203
439938,737,observation_94,0.011056
439939,737,observation_171,0.012660
439940,737,condition_2375,0.024597
...,...,...,...
439967,737,condition_272,0.198860
439968,737,drug_335,0.200959
439969,737,measurement_858,0.211757
439970,737,measurement_355,0.224878


727 Vaatlust kokku
vaatluseid surmadega on 263

In [2]:
pivot_table = data.pivot_table(index='SUBJECT_ID', columns='DEFINITION_ID', values='TIME', aggfunc='mean')
# Reset the index to have 'SUBJECT_ID' as a regular column
pivot_table.reset_index(inplace=True)
pivot_table

DEFINITION_ID,SUBJECT_ID,condition_1,condition_10,condition_100,condition_1000,condition_1001,condition_1002,condition_1003,condition_1004,condition_1005,...,procedure_90,procedure_91,procedure_92,procedure_93,procedure_94,procedure_95,procedure_96,procedure_97,procedure_98,procedure_99
0,1,2.096877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.893354,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.46837,NaN,...,2.483872,NaN,2.525181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.80732,NaN,NaN,NaN,NaN,NaN
4,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pivot_table["death"].notna().sum()

263

In [9]:
death_data = pivot_table[pivot_table['death'].notnull()]
alive_data = pivot_table[pivot_table['death'].isnull()]


In [10]:
death_summary = death_data.describe()
alive_summary = alive_data.describe()

In [11]:
alive_summary

DEFINITION_ID,SUBJECT_ID,condition_1,condition_10,condition_100,condition_1000,condition_1001,condition_1002,condition_1003,condition_1004,condition_1005,...,procedure_90,procedure_91,procedure_92,procedure_93,procedure_94,procedure_95,procedure_96,procedure_97,procedure_98,procedure_99
count,464.000000,45.000000,47.000000,48.000000,41.000000,38.000000,45.000000,37.000000,38.000000,35.000000,...,64.000000,53.000000,64.000000,61.000000,67.000000,73.000000,62.000000,63.000000,71.000000,62.000000
mean,492.377155,4.733160,4.324227,4.328446,4.211708,3.917639,4.300284,4.123906,3.774794,4.305106,...,3.865682,4.302589,4.045982,4.114831,4.032966,3.717923,4.001214,4.207902,4.298302,4.353235
std,281.881257,2.464101,2.369990,2.426591,2.241243,2.116701,2.195921,2.448136,2.519933,2.408458,...,2.486257,2.048758,2.185328,2.267984,2.541374,2.370525,2.383833,2.392755,2.131334,2.769074
min,1.000000,0.022541,0.006989,0.028453,0.015860,0.628211,0.018503,0.015747,0.032077,0.001293,...,0.004873,0.267114,0.004254,0.467331,0.022095,0.076302,0.023931,0.005151,0.322160,0.016928
25%,241.500000,2.528556,2.780455,2.662129,2.346021,2.268800,2.626600,1.976623,1.282468,2.686727,...,1.993098,3.054118,2.365315,2.215323,1.634278,1.503579,2.039948,2.451303,2.821374,1.880033
50%,488.500000,5.290974,4.647509,4.240921,4.193247,3.573758,4.832573,4.478329,3.584575,4.328816,...,3.540337,4.432169,4.220080,4.125568,4.076217,3.703906,3.927460,4.181442,4.460856,4.282035
75%,723.250000,6.664325,5.890427,6.380657,5.667011,5.529539,5.965210,6.102914,5.729755,6.216072,...,6.063934,6.010811,5.556534,5.974841,6.105360,5.847023,6.055098,6.175342,5.740056,6.975824
max,984.000000,10.950420,10.220919,9.815834,8.937586,7.742380,7.762194,8.138636,9.593218,7.452807,...,9.575719,7.629409,7.782783,8.872227,9.651114,7.844500,7.966591,9.934803,9.853555,11.669065


In [12]:
death_summary

DEFINITION_ID,SUBJECT_ID,condition_1,condition_10,condition_100,condition_1000,condition_1001,condition_1002,condition_1003,condition_1004,condition_1005,...,procedure_90,procedure_91,procedure_92,procedure_93,procedure_94,procedure_95,procedure_96,procedure_97,procedure_98,procedure_99
count,263.000000,8.000000,14.000000,11.000000,14.000000,13.000000,6.000000,10.000000,19.000000,14.000000,...,19.000000,15.000000,18.000000,15.000000,21.000000,18.000000,16.000000,17.000000,15.000000,16.000000
mean,494.828897,2.553079,2.934783,2.241809,2.131820,2.086976,3.631096,2.088183,2.404648,2.362607,...,2.493329,1.441075,2.980514,2.406902,2.907373,1.939551,1.995885,2.314934,2.323402,2.666518
std,277.570480,1.977122,1.989197,1.374221,1.840517,1.367946,1.359663,1.957649,1.991821,1.873011,...,1.906337,1.500676,2.057634,1.935891,2.125116,1.702365,1.306962,1.958220,2.817163,2.631063
min,2.000000,0.154999,0.113331,0.627414,0.241683,0.082528,1.489511,0.000013,0.019747,0.002785,...,0.025589,0.001714,0.028304,0.001498,0.009456,0.010984,0.014898,0.012074,0.059363,0.041517
25%,272.000000,1.309277,1.312091,0.856173,0.834314,0.937487,3.036945,0.533285,0.607130,1.310542,...,0.603939,0.170823,1.613640,0.661120,0.811290,0.611104,0.905282,0.233222,0.842235,0.543382
50%,498.000000,2.091663,2.835150,2.108238,1.589618,2.172736,3.764393,1.657215,2.377875,1.840597,...,2.425020,0.828341,2.564661,2.152591,2.807320,1.505260,1.590488,1.948864,1.513521,1.474376
75%,734.500000,3.942775,4.447695,3.242037,2.875539,3.031475,4.394502,2.965225,3.642385,3.313260,...,3.582835,2.534632,4.542671,4.138275,4.347233,2.864975,3.066403,3.812569,2.658555,5.390578
max,981.000000,5.568918,6.439186,4.385009,5.897697,4.651146,5.369277,5.928825,6.779928,6.185741,...,6.296863,4.394856,6.240791,5.349776,6.386735,5.874608,4.112006,5.632452,11.320301,7.360477


In [ ]:
mean_diff = death_summary.mean() - alive_summary.mean()


In [ ]:
mean_diff.sort_values(ascending = False)

In [ ]:
alive_summary["observation_110"]

In [ ]:
death_summary["observation_110"]